In [ ]:
import numpy as np
from numpy.testing import assert_allclose

from qiskit import QiskitError
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.circuit.library import HGate, QFT

from qiskit.quantum_info.random import random_unitary, random_density_matrix, random_pauli
from qiskit.quantum_info.states import DensityMatrix, Statevector
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or statevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def rand_rho(n):
        """Return random pure state density matrix"""
        rho = rand_vec(n, normalize=True)
        return np.outer(rho, np.conjugate(rho))

def assertEqual(a, b):
    assert a == b

def assertTrue(a):
    assert a == True

def assertFalse(a):
    assert a == False

def assertRaises(exception, func, *args):
    try:
        func(*args)
    except exception:
        pass

In [ ]:
print(np.sum([np.abs(a)**2 for a in rand_vec(3, True)]))
print(np.trace(rand_rho(3)))

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
rho = rand_rho(8)
for dims in [None, 8]:
    state = DensityMatrix(rho, dims=dims)
    assert_allclose(state.data, rho)
    assert state.dim == 8
    assert state.dims() == (2, 2, 2)
    assert state.num_qubits == 3

In [ ]:
"""Test initialization from array."""
rho = rand_rho(3)
state = DensityMatrix(rho)
assert_allclose(state.data, rho)
assert state.dim == 3
assert state.dims() == (3,)
assert state.num_qubits == None

rho = rand_rho(2 * 3 * 4)
state = DensityMatrix(rho, dims=[2, 3, 4])
assert_allclose(state.data, rho)
assert state.dim == 2 * 3 * 4
assert state.dims() == (2, 3, 4)
assert state.num_qubits == None

In [ ]:
"""Test initialization exception from array."""
rho = rand_rho(4)
try:
    DensityMatrix(rho, dims=[4, 2])
except QiskitError:
    pass
try:
    DensityMatrix(rho, dims=[2, 4])
except QiskitError:
    pass
try:
    DensityMatrix(rho, dims=5)
except QiskitError:
    pass

In [ ]:
"""Test initialization from DensityMatrix."""
rho1 = DensityMatrix(rand_rho(4))
rho2 = DensityMatrix(rho1)
assert rho1 == rho2

In [ ]:
 """Test initialization from DensityMatrix."""
vec = rand_vec(4)
target = DensityMatrix(np.outer(vec, np.conjugate(vec)))
rho = DensityMatrix(Statevector(vec))
assert rho == target

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])
target_vec = Statevector(np.kron(u1, u0).dot([1, 0, 0, 0]))
target = DensityMatrix(target_vec)
rho = DensityMatrix(circ)
assert rho == target

# Test tensor product of 1-qubit gates
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.x(1)
circuit.ry(np.pi / 2, 2)
target = DensityMatrix.from_label("000").evolve(Operator(circuit))
rho = DensityMatrix(circuit)
assert rho == target

# Test decomposition of Controlled-Phase gate
lam = np.pi / 4
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.h(1)
circuit.cp(lam, 0, 1)
target = DensityMatrix.from_label("00").evolve(Operator(circuit))
rho = DensityMatrix(circuit)
assert rho == target

In [ ]:
"""Test initialization from a circuit."""
# random unitaries
u0 = random_unitary(2).data
u1 = random_unitary(2).data
# add to circuit
qr = QuantumRegister(2)
circ = QuantumCircuit(qr)
circ.unitary(u0, [qr[0]])
circ.unitary(u1, [qr[1]])

# Test decomposition of controlled-H gate
circuit = QuantumCircuit(2)
circ.x(0)
circuit.ch(0, 1)
target = DensityMatrix.from_label("00").evolve(Operator(circuit))
rho = DensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

# Test initialize instruction
init = Statevector([1, 0, 0, 1j]) / np.sqrt(2)
target = DensityMatrix(init)
circuit = QuantumCircuit(2)
circuit.initialize(init.data, [0, 1])
rho = DensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

# Test reset instruction
target = DensityMatrix([1, 0])
circuit = QuantumCircuit(1)
circuit.h(0)
circuit.reset(0)
rho = DensityMatrix.from_instruction(circuit)
assertEqual(rho, target)

In [ ]:
"""Test initialization from an instruction."""
target_vec = Statevector(np.dot(HGate().to_matrix(), [1, 0]))
target = DensityMatrix(target_vec)
rho = DensityMatrix.from_instruction(HGate())
assertEqual(rho, target)

In [ ]:
"""Test initialization from a label"""
x_p = DensityMatrix(np.array([[0.5, 0.5], [0.5, 0.5]]))
x_m = DensityMatrix(np.array([[0.5, -0.5], [-0.5, 0.5]]))
y_p = DensityMatrix(np.array([[0.5, -0.5j], [0.5j, 0.5]]))
y_m = DensityMatrix(np.array([[0.5, 0.5j], [-0.5j, 0.5]]))
z_p = DensityMatrix(np.diag([1, 0]))
z_m = DensityMatrix(np.diag([0, 1]))

label = "0+r"
target = z_p.tensor(x_p).tensor(y_p)
assertEqual(target, DensityMatrix.from_label(label))

label = "-l1"
target = x_m.tensor(y_m).tensor(z_m)
assertEqual(target, DensityMatrix.from_label(label))

In [ ]:
"""Test __eq__ method"""
for _ in range(10):
    rho = rand_rho(4)
    assertEqual(DensityMatrix(rho), DensityMatrix(rho.tolist()))

In [ ]:
"""Test DensityMatrix copy method"""
for _ in range(5):
    rho = rand_rho(4)
    orig = DensityMatrix(rho)
    cpy = orig.copy()
    cpy._data[0] += 1.0
    assertFalse(cpy == orig)

In [ ]:
"""Test is_valid method."""
state = DensityMatrix(np.eye(2))
assertFalse(state.is_valid())
for _ in range(10):
    state = DensityMatrix(rand_rho(4))
    assertTrue(state.is_valid())

In [ ]:
"""Test to_operator method for returning projector."""
for _ in range(10):
    rho = rand_rho(4)
    target = Operator(rho)
    op = DensityMatrix(rho).to_operator()
    assertEqual(op, target)

In [ ]:
"""Test evolve method for operators."""
for _ in range(10):
    op = random_unitary(4)
    rho = rand_rho(4)
    target = DensityMatrix(np.dot(op.data, rho).dot(op.adjoint().data))
    evolved = DensityMatrix(rho).evolve(op)
    assertEqual(target, evolved)

In [ ]:
"""Test subsystem evolve method for operators."""
# Test evolving single-qubit of 3-qubit system
for _ in range(5):
    rho = rand_rho(8)
    state = DensityMatrix(rho)
    op0 = random_unitary(2)
    op1 = random_unitary(2)
    op2 = random_unitary(2)

    # Test evolve on 1-qubit
    op = op0
    op_full = Operator(np.eye(4)).tensor(op)
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0]), target)

    # Evolve on qubit 1
    op_full = Operator(np.eye(2)).tensor(op).tensor(np.eye(2))
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[1]), target)

    # Evolve on qubit 2
    op_full = op.tensor(np.eye(4))
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2]), target)

    # Test evolve on 2-qubits
    op = op1.tensor(op0)

    # Evolve on qubits [0, 2]
    op_full = op1.tensor(np.eye(2)).tensor(op0)
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0, 2]), target)

    # Evolve on qubits [2, 0]
    op_full = op0.tensor(np.eye(2)).tensor(op1)
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2, 0]), target)

    # Test evolve on 3-qubits
    op = op2.tensor(op1).tensor(op0)

    # Evolve on qubits [0, 1, 2]
    op_full = op
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[0, 1, 2]), target)

    # Evolve on qubits [2, 1, 0]
    op_full = op0.tensor(op1).tensor(op2)
    target = DensityMatrix(np.dot(op_full.data, rho).dot(op_full.adjoint().data))
    assertEqual(state.evolve(op, qargs=[2, 1, 0]), target)

In [ ]:
"""Test conjugate method."""
for _ in range(10):
    rho = rand_rho(4)
    target = DensityMatrix(np.conj(rho))
    state = DensityMatrix(rho).conjugate()
    assertEqual(state, target)

In [ ]:
"""Test expand method."""
for _ in range(10):
    rho0 = rand_rho(2)
    rho1 = rand_rho(3)
    target = np.kron(rho1, rho0)
    state = DensityMatrix(rho0).expand(DensityMatrix(rho1))
    assertEqual(state.dim, 6)
    assertEqual(state.dims(), (2, 3))
    assert_allclose(state.data, target)

In [ ]:
"""Test tensor method."""
for _ in range(10):
    rho0 = rand_rho(2)
    rho1 = rand_rho(3)
    target = np.kron(rho0, rho1)
    state = DensityMatrix(rho0).tensor(DensityMatrix(rho1))
    assertEqual(state.dim, 6)
    assertEqual(state.dims(), (3, 2))
    assert_allclose(state.data, target)

In [ ]:
"""Test add method."""
for _ in range(10):
    rho0 = rand_rho(4)
    rho1 = rand_rho(4)
    state0 = DensityMatrix(rho0)
    state1 = DensityMatrix(rho1)
    assertEqual(state0 + state1, DensityMatrix(rho0 + rho1))

In [ ]:
"""Test add method raises exceptions."""
state1 = DensityMatrix(rand_rho(2))
state2 = DensityMatrix(rand_rho(3))
assertRaises(QiskitError, state1.__add__, state2)

In [ ]:
"""Test subtract method."""
for _ in range(10):
    rho0 = rand_rho(4)
    rho1 = rand_rho(4)
    state0 = DensityMatrix(rho0)
    state1 = DensityMatrix(rho1)
    assertEqual(state0 - state1, DensityMatrix(rho0 - rho1))

In [ ]:
"""Test multiply method."""
for _ in range(10):
    rho = rand_rho(4)
    state = DensityMatrix(rho)
    val = np.random.rand() + 1j * np.random.rand()
    assertEqual(val * state, DensityMatrix(val * state))

In [ ]:
"""Test negate method"""
for _ in range(10):
    rho = rand_rho(4)
    state = DensityMatrix(rho)
    assertEqual(-state, DensityMatrix(-1 * rho))